In [1]:
import numpy as np
from qibo import models, set_backend, gates
from qibo.symbols import X,Y,Z
from qibo.hamiltonians import SymbolicHamiltonian
set_backend("qibojit")

two_to_one = lambda u,v: int(u+num_cities*v)

def tsp_phaser(distance_matrix):
    num_cities = distance_matrix.shape[0]
    form = 0
    for u in range(num_cities):
        for v in range(num_cities):
            form += distance_matrix[u, v]* np.sum([Z(two_to_one(u, i))*Z(two_to_one(v,(i + 1)%num_cities)) for i in range(num_cities)])
    ham = SymbolicHamiltonian(form)
    return ham


def tsp_mixer(num_cities, ):
    splus = lambda u, i: X(two_to_one(u, i)) + 1j * Y(two_to_one(u, i))
    sminus = lambda u, i: X(two_to_one(u, i)) - 1j * Y(two_to_one(u, i))
    form = 0
    for i in range(num_cities):
        for u in range(num_cities):
            for v in range(num_cities):
                    form += splus(u, i) * splus(v, (i + 1) % num_cities) * sminus(u, (i + 1) % num_cities
                    ) * sminus(v, i) + sminus(u, i) * sminus(
                        v, (i + 1) % num_cities
                    ) * splus(
                        u, (i + 1) % num_cities
                    ) * splus(
                        v, i
                    )
    ham = SymbolicHamiltonian(form.expand())
    return ham

def prepare_initial_state(ordering):
        """
        To run QAOA by Hadsfield, we need to start from a valid permutation function to ensure feasibility.

        Args:
            ordering (array): A list describing permutation from 0 to n-1

        Returns:
            An initial state that is used to start TSP QAOA.

        """
        c = models.Circuit(len(ordering) ** 2)
        for i in range(len(ordering)):
            c.add(gates.X(two_to_one(ordering[i], i)))
        return c().state()

[Qibo 0.2.11|INFO|2024-10-24 11:47:20]: Using qibojit (numba) backend on /CPU:0


In [10]:
distance_matrix = np.array([[0, 0.1, 1], [2, 0, 0.2],[0.3, 3, 0]])
num_cities = distance_matrix.shape[0]
nlayers = 2
initial_state = prepare_initial_state([0,2,1])
qaoa = models.QAOA(tsp_phaser(distance_matrix), mixer=tsp_mixer(num_cities))
best_energy, final_parameters, extra = qaoa.minimize(0.1 * np.random.random(2*nlayers),initial_state=initial_state)

In [11]:
from qibo import gates
nqubits = int(num_cities**2)
circuit = models.Circuit(nqubits)
circuit.add(gates.M(*range(nqubits)))
result = circuit(initial_state=qaoa.execute(initial_state), nshots=10000)
freq_counter = result.frequencies()
freq_counter

Counter({'100010001': 8678,
         '010001100': 863,
         '001100010': 457,
         '001010100': 2})

In [12]:
def convert_to_standard_Cauchy(config):
    m = int(np.sqrt(len(config)))
    cauchy = [-1] * m  # Cauchy's notation for permutation, e.g. (1,2,0) or (2,0,1)
    for i in range(m):
        for j in range(m):
            if config[m * i + j] == '1':
                cauchy[j] = i  # citi i is in slot j
    for i in range(m):
        if cauchy[i] == 0:
            cauchy = cauchy[i:] + cauchy[:i]
            return tuple(cauchy)  # now, the cauchy notation for permutation begins with 0
    
    
def evaluate_dist(cauchy):
    '''
    Given a permutation of 0 to n-1, we compute the distance of the tour
    
    '''
    m = len(cauchy)
    return sum(distance_matrix[cauchy[i]][cauchy[(i+1)%m]] for i in range(m))